In [299]:
from lxml import etree
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.collections import PatchCollection
from IPython.html.widgets import *
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import requests
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import dateutil
from datetime import date
import jinja2
from IPython.display import *
import folium

In [300]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [28]:
from dateutils

SyntaxError: invalid syntax (<ipython-input-28-99dcb6538c3d>, line 1)

In [126]:
def display_earthquake_markers_map(latitude, longitude, feed, map_type, div):
    # Make a request to the supplied feed and parse the response as JSON
    response = requests.get(feed)
    response_json = response.json()

    # Create a DataFrame from a list comprehension built from the "features"
    # field of the JSON response.
    latest_earthquakes = [(
        date.fromtimestamp(float(feature['properties']['time']) / 1000.0),
        feature['geometry']['coordinates'][1],
        feature['geometry']['coordinates'][0],
        feature['properties']['mag'],
        feature['properties']['title'],
    ) for feature in response_json['features']]

    latest_earthquakes_df = pd.DataFrame(latest_earthquakes, columns=['time', 'latitude', 'longitude', 'mag', 'title'])
    latest_earthquakes_df.set_index('time', inplace=True)

    # Define a Jinja2 template that creates the map, and iterates of the
    # features to create a marker for each of them.
    latest_earthquakes_javascript = jinja2.Template(
    """
 require(['async!http://maps.googleapis.com/maps/api/js?sensor=false&libraries=visualization&callback=console.log'], function() {
 var mapOptions = {
 zoom: 2.0,
 center: new google.maps.LatLng({{ latitude }}, {{ longitude }}),
 mapTypeId: {{ map_type }}
 };

 var map = new google.maps.Map(document.getElementById('latest_earthquakes_map_markers'), mapOptions);
 var markers = Array();

 var machinalis_marker = new google.maps.Marker({
 position: new google.maps.LatLng(-31.4122599,-64.2010031),
 icon: {
 size: new google.maps.Size(30, 30),
 url: 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAB4AAAAeCAYAAAA7MK6iAAAABmJLR0QA/wD/AP+gvaeTAAAACXBIWXMAAAsTAAALEwEAmpwYAAAAB3RJTUUH3gUdESIC2is5hgAAAB1pVFh0Q29tbWVudAAAAAAAQ3JlYXRlZCB3aXRoIEdJTVBkLmUHAAAF9klEQVRIx72WaVBTVxTH/29LYgCxskQkLlSgSFSMYqmgVRSjIoxUpVrHqrWiWMcFl7rUDrU6Mu1UaV1ah6mOSGHccBxrnSq4VS1QQcRQjIIgiAUEFRASs7x3+yESQxIRmE7vpzfv3nt+95zzP+deoBujMHbDyjyXcJLLhRF1WFxVUKCC6aoNujtg44073xG9AaAptObf6bdBOlj5v4BB220j/zn4aMTCThkqW7TN8p2lWvLG9eybFszOPoSLk+PneurI96VyacAHGclPHa3zPfglyhZtO1L74KHrmPMpUzvt8TpXhd3keP8hLjcDPix2unYnvSWn2IPTGvp0ZExbVj2BuaKeku8dpS349KulnQJv3vJFv0ynkHX3P/t2FACkKqcHJlOBzfqyagUx8QBNARTVYS4pmjZSNA1TQ2MPY+r5/fkRy1YDwG9LtsjOeIQvvTAsdjIAJPmHvwq1Zmfqr95GLqj+wGkURq46T0qrw/VV/4Biuqc/MDRMl24lF83ZFG04lR/ONxkpmmpGkn84t+neJRMNAJkTFkYxrYYgUOY9+qx8laGylrMYEQjY3j1hchY3dsTiPHrVEkGwUhADXeaVCXxjCwUKEJ49h8pzkNoSapahtbSIA9o2mcNqgdKyt1pefDJpcEzazprXQZtX7UHA8aSRxM/7D7vSo8wFR7EMOIm4wQKennXwYtFQd4VkuH+Tnex794Ru4UTlmB0Jmo687fnDCgBAqObEOOfRQ0vsIu/qBFHs+MRh2fvGthPX4ssZJS5BfhG2GwR/73XjtieUdTa1pQu24oVvn2lEaK/DZi/XA8rD27522ECqLuQEt1Mpx0IX+s4vu5VRndaUX2oign9OfCCdFFwIq3w7CdT8dpG8MWNtnFBcsRsA6GYdx1sJShoSWBXyzfo6O+vWAnp5Rtsf1yrvJo2SiI8JBqN5QUUtdyNglk4QBFByz2ssybqZQvQGAAAPysoEgUjued1hvfaXPUJV/UBQAOvmihYJ1WCXUxfpaUpCgJdgYuLBl9dIAIBUPo5ghRfmW8ZxfbB25VM9dyvkGYk+l2clLAdPJOWc4eySYz8+sFN5D1ZvZ7etUggBK1X612mLSmWOuPyz58Nt/8kzEs3t9ETyvg5V3qwfBiPv+ILw6GVi97o09JuslI8w8bxxQCvicL8mvu102rySYHRzKOQDlvHqAouXEp++KB/QU0GaWpy1Ura2XSzygj9WEXXFuVe3LAGJUE4cfXbvxc4CT0XFg14zQySP2681PKxj2sCtPp45EzUnQx2WE9PXbYptTqTPdOld8TTmzH64f354gaH6sQUKAJ4eHrLEoRNk7cApo2d6F/jOqDedy0+wNaTLv9MnJ2jOGQC4t3j7a4Gno8wZuh67fjZXXpdiO9+So347qoarLfloc5IFLGOlK0xVde7WIbbutfTdR9PU4cuuNfagvayN7QpUWb41q6ewuaMXxosu3DrCt+pga4tiaPDNLTDcrlhvpW/gr76RrcKTZikAcHIPo+FhPUfZVgPLQqwKzr6iKdrxgqOvbrh9zrQrUDUoOEAR51JcvUZfWcO1bzQEIm8P3lDTwJjhDO4rvebPvX4k7RVYtXy+UNNwsMmg31gTM+KnIXnVCaar6m1gGbuuRTv1AC0RARQFYuTBP281e2Z9UkJg7OdedjXSb+TMevHwJ+p76bSztOi9vNSojLA5cNg5dgZEYK0mGwWRK4uNWfkKB6/KjgcBmF7OpHJeaB9xYXl99KVD5I09tm2cmbt2oOfZWxWCTt/dUoYwZMDs0IL0Y1163nrerVtkgRICTtYbpsD+JxgfL1A2yRfLZTD6epVQwwf9bi1MsdZ4tMvv6kadVsO4OJlvIoaBJDp0/pjCjNhRmuMU3dfNaA24O8Tdf+zfxxUhealTaTfXVggCKJoGLxXf7DJYVXIyI7e/yJULDngkCVVkK/ZtTHulm/YpM9U+dbb06JixCU4jAwzVo/qvebcgbWSX87NnRPRr5wrGL/0zVzKW5IrCSMHAGENKWKzc0brM9+e91sa/SYtOrUfMruYAAAAASUVORK5CYII='
 },
 title: 'http://www.machinalis.com',
 map: map
 });

 {% for earthquake in latest_earthquakes %}
 markers.push(new google.maps.Marker({
 position: new google.maps.LatLng({{ earthquake[1] }}, {{ earthquake[2] }}),
 title: "{{ earthquake[4] }}",
 map: map
 }));
 {% endfor %}
 });
 """
    )

    # Render the template and display it as Javascript
    IPython.display.YouTubeVideo('sjfsUzECqK0')

In [132]:
interact(
    display_earthquake_markers_map,
    latitude=widgets.FloatText(description='Latitude', value=-31.4122599, min=-90.0, max=90.0),
    longitude=widgets.FloatText(description='Longitude', value=-64.2010031, min=-180.0, max=180.0),
    map_type=widgets.Dropdown(
        description="Map Type", 
        options={'Road Map': 'google.maps.MapTypeId.ROADMAP','Satellite': 'google.maps.MapTypeId.SATELLITE','Hybrid': 'google.maps.MapTypeId.HYBRID','Terrain': 'google.maps.MapTypeId.TERRAIN'}, 
        value='google.maps.MapTypeId.SATELLITE',
    ),
    feed=widgets.Dropdown(
        description='Feed', 
        options={
            'Significant Earthquakes': 'http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_week.geojson',
            'M4.5+ Earthquakes': 'http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_week.geojson',
            'M2.5+ Earthquakes': 'hytp://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson',
            'M1.0+ Earthquakes': 'http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/1.0_week.geojson',
            'All Earthquakes': 'http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson'
        },
        value='http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_week.geojson',
    ),
    div=widgets.HTML(value='<div id="latest_earthquakes_map_markers" style="width: 800px; height: 600px"></div>')
);

In [288]:
from IPython.display import Javascript

Javascript("""$.getScript('http://d3js.org/d3.v3.min.js')""")



<IPython.core.display.Javascript object>

In [65]:
from IPython.display import display_html
display_html(raw=www.sheldonbrown.com/web_sample1.html)

NameError: name 'www' is not defined

In [134]:


from fastkml.kml import KML

def read_kml(fname='ss.kml'):
    kml = KML()
    kml.from_string(open(fname).read())
    points = dict()
    for feature in kml.features():
        for placemark in feature.features():
            if placemark.styleUrl.startswith('#hf'):
                points.update({placemark.name:
                            (placemark.geometry.y, placemark.geometry.x, )})
    return points



In [305]:
df=pd.read_csv("./all_05_14.csv")

In [306]:
import ast
datadict = {}
locations = []
descriptions = []
counts = []
for index, songs in df.iterrows():
    location = ast.literal_eval(songs['artist_location'])
    year = songs['year']
    artist = songs['artist_name']
    artistlink = songs['url']
    song = songs['song']
    songlink = songs['songurl']
    if not location:
        continue
    latitude = location['latitude']
    longitude = location['longitude']
    city = location['location']
    if [latitude, longitude] not in locations:
        locations.append([latitude, longitude])
        descriptions.append(
            '<p><strong>%s</strong></p>' % city
            + '<br>'
            + '<p><a href="https://en.wikipedia.org%s">%s</a>' % (songlink, song)
            + ', <a href="https://en.wikipedia.org%s">%s</a>' % (artistlink, artist)
            + ' (<a href="https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_%s">%s</a>)' % (year, year)
        )
        counts.append(1)
    else:
        pos = locations.index([latitude, longitude])
        description = descriptions.pop(pos)
        descriptions.insert(
            pos, 
            '%s' % description
            + '<p><a href="https://en.wikipedia.org%s">%s</a>' % (songlink, song)
            + ', <a href="https://en.wikipedia.org%s">%s</a>' % (artistlink, artist)
            + ' (<a href="https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_%s">%s</a>)' % (year, year)
        )
        count = counts.pop(pos)
        counts.insert(pos, 1+count)
datadict['locations'] = locations
datadict['descriptions'] = descriptions
datadict['counts'] = counts
newdf = pd.DataFrame(datadict)
newdf.head()

,counts,descriptions,locations
0,32,"<p><strong>New York, NY</strong></p><br><p><a ...","[40.714269, -74.005973]"
1,5,"<p><strong>Fullerton, California</strong></p><...","[33.870292, -117.925338]"
2,2,"<p><strong>Baltimore, MD</strong></p><br><p><a...","[39.296536, -76.623489]"
3,13,"<p><strong>Burleson, TX</strong></p><br><p><a ...","[32.492683, -97.264359]"
4,60,"<p><strong>Atlanta, Georgia</strong></p><br><p...","[33.844371, -84.47405]"


In [308]:
import folium


def inline_map(m, width=1000, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed


width, height = 1000, 500
radars = folium.Map(location=[40, -122], zoom_start=2,
                    tiles='OpenStreetMap', width=width, height=height)

for index, place in newdf.iterrows():
    radars.circle_marker(place['locations'], popup=place['descriptions'], radius=3000*place['counts'], fill_opacity=0.1)

inline_map(radars)

In [241]:
fname = './ss.kml'
locations = read_kml(fname)

In [170]:
import folium


def inline_map(m, width=1000, height=750):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed


width, height = 1000, 750
radars = folium.Map(location=[40, -122], zoom_start=2,
                    tiles='OpenStreetMap', width=width, height=height)

for name, location in locations.items():
    radars.simple_marker(location=location, popup=name)

inline_map(radars)